In [1]:
from gnatss.ops.posfilter import kalman_filtering
from gnatss.loaders import read_novatel_L1_data_files
from gnatss.loaders import load_gps_solutions, load_travel_times
from gnatss.ops.data import clean_tt, preprocess_tt

import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)


import warnings
warnings.filterwarnings('ignore')

In [2]:
travel_time_files = ['/home/madhavmk/Desktop/offshore-geodesy/tests/data/2022/NCL1/208/WG_20220727/pxp_tt', '/home/madhavmk/Desktop/offshore-geodesy/tests/data/2022/NCL1/209/WG_20220728/pxp_tt', '/home/madhavmk/Desktop/offshore-geodesy/tests/data/2022/NCL1/210/WG_20220729/pxp_tt']

transponder_ids = ['NCL1-1', 'NCL1-2', 'NCL1-3']
travel_times_correction = 0.0
transducer_delay_time = 0.0

all_travel_times = load_travel_times(
        files=travel_time_files, transponder_ids=transponder_ids
    )


cleaned_travel_times = clean_tt(all_travel_times, transponder_ids, travel_times_correction, transducer_delay_time)
twtt_df = preprocess_tt(cleaned_travel_times)
# twtt = twtt_df.rename(columns={'time': 'dts'})
twtt_df

,time,MT,TT,ST
0,712215465.00000,ST,0.00000,NaN
1,712215467.26326,NCL1-1,2.26326,712215465.00000
2,712215467.38340,NCL1-2,2.38340,712215465.00000
3,712215467.40082,NCL1-3,2.40082,712215465.00000
4,712215480.00000,ST,0.00000,NaN
...,...,...,...,...
34407,712396373.47335,NCL1-3,2.47334,712396371.00000
34408,712396386.00000,ST,0.00000,NaN
34409,712396388.25092,NCL1-1,2.25093,712396386.00000
34410,712396388.31850,NCL1-2,2.31850,712396386.00000


In [3]:
inspvaa_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")
insstdeva_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'],
                                          data_format="INSSTDEVA")

In [4]:
inspvaa_df

,Week,Seconds,lat,lon,alt,ant_n,ant_e,ant_u,roll,pitch,heading,time
0,2220,400732.75000,45.30228,-124.96597,-28.34490,-0.68580,0.10140,0.24130,-3.55017,1.67569,225.48596,712293532.75000
1,2220,400732.80000,45.30228,-124.96597,-28.33400,-0.63800,0.07880,0.27340,-3.54066,1.37273,225.57350,712293532.80000
2,2220,400732.85000,45.30228,-124.96597,-28.32250,-0.58330,0.05340,0.29260,-3.18302,1.12580,225.64909,712293532.85000
3,2220,400732.90000,45.30228,-124.96597,-28.31040,-0.54400,0.04050,0.29500,-2.58513,0.94125,225.70457,712293532.90000
4,2220,400732.95000,45.30228,-124.96597,-28.29900,-0.51220,0.03460,0.28010,-1.81905,0.82661,225.73288,712293532.95000
...,...,...,...,...,...,...,...,...,...,...,...,...
3865082,2220,469048.15000,45.30206,-124.96584,-23.49010,-0.97260,1.06310,-0.11580,2.40545,-1.05323,127.18168,712361848.15000
3865083,2220,469048.20000,45.30206,-124.96584,-23.49620,-0.98040,1.08850,-0.14160,2.84893,-0.73574,126.52136,712361848.20000
3865084,2220,469048.25000,45.30205,-124.96584,-23.50370,-0.97090,1.12550,-0.17280,3.10908,-0.44089,125.87306,712361848.25000
3865085,2220,469048.30000,45.30205,-124.96584,-23.51290,-0.95420,1.15780,-0.20870,3.18417,-0.16181,125.24787,712361848.30000


In [5]:
insstdeva_df

,Week,Seconds,lat std,lon std,alt std,ant_n std,ant_e std,ant_u std,roll_roll,pitch_pitch,heading_heading,Ext sol stat,Time Since Update,time
0,2220,400732.75000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.75000
1,2220,400732.80000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.80000
2,2220,400732.85000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.85000
3,2220,400732.90000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.90000
4,2220,400732.95000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.95000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865303,2220,469048.15000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.15000
3865304,2220,469048.20000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.20000
3865305,2220,469048.25000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.25000
3865306,2220,469048.30000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.30000


In [6]:
gps_novatel_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/209/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/210/GPS_PPP/GPS_POS_FREED',
    ],
    is_l1_data=True,
)
gps_novatel_df

,time,x,y,z,sdx,sdy,sdz
0,712207183.00000,-2574893.32050,-3681574.16020,4512096.64040,2.44910,2.44890,2.44880
1,712207184.00000,-2574893.27980,-3681574.05620,4512096.74780,2.23590,2.23570,2.23560
2,712207185.00000,-2574893.23910,-3681573.95220,4512096.85520,2.00000,1.99980,1.99970
3,712207186.00000,-2574893.19830,-3681573.84820,4512096.96260,1.73230,1.73200,1.73190
4,712207187.00000,-2574893.15760,-3681573.74410,4512097.07000,1.41470,1.41440,1.41420
...,...,...,...,...,...,...,...
193288,712400473.00000,-2575282.93430,-3682610.17990,4511037.84700,0.01340,0.01370,0.01580
193289,712400474.00000,-2575283.67510,-3682610.14460,4511038.07690,0.01340,0.01370,0.01580
193290,712400475.00000,-2575283.93220,-3682610.22220,4511037.95340,0.01340,0.01370,0.01580
193291,712400476.00000,-2575283.70300,-3682609.18440,4511037.48930,0.01340,0.01370,0.01580


In [10]:
kalman_filtered_df, x, P = kalman_filtering(twtt_df, inspvaa_df, insstdeva_df, gps_novatel_df)

In [11]:
kalman_filtered_df

,x,y,z,xx,yy,zz,time
0,-2574898.37352,-3681585.48281,4512084.51296,0.00042,0.00029,0.00024,712207205.00000
1,-2574898.38786,-3681585.51719,4512084.44755,0.00040,0.00027,0.00021,712207205.05000
2,-2574898.40220,-3681585.55159,4512084.38215,0.00039,0.00026,0.00020,712207205.10000
3,-2574898.41604,-3681585.58631,4512084.31729,0.00039,0.00025,0.00020,712207205.15000
4,-2574898.42937,-3681585.62138,4512084.25338,0.00038,0.00024,0.00019,712207205.20000
...,...,...,...,...,...,...,...
3890870,-2575284.79106,-3682607.96396,4511038.02327,0.00007,0.00007,0.00009,712400477.50000
3890871,-2575284.85003,-3682607.93236,4511038.03698,0.00007,0.00007,0.00009,712400477.55000
3890872,-2575284.90925,-3682607.89903,4511038.04687,0.00007,0.00008,0.00009,712400477.60000
3890873,-2575284.96746,-3682607.86363,4511038.05163,0.00007,0.00008,0.00009,712400477.65000


In [12]:
kalman_filtered_df = kalman_filtered_df[
    ['time', 'x', 'y', 'z', 'xx', 'yy', 'zz']
]
kalman_filtered_df

,time,x,y,z,xx,yy,zz
0,712207205.00000,-2574898.37352,-3681585.48281,4512084.51296,0.00042,0.00029,0.00024
1,712207205.05000,-2574898.38786,-3681585.51719,4512084.44755,0.00040,0.00027,0.00021
2,712207205.10000,-2574898.40220,-3681585.55159,4512084.38215,0.00039,0.00026,0.00020
3,712207205.15000,-2574898.41604,-3681585.58631,4512084.31729,0.00039,0.00025,0.00020
4,712207205.20000,-2574898.42937,-3681585.62138,4512084.25338,0.00038,0.00024,0.00019
...,...,...,...,...,...,...,...
3890870,712400477.50000,-2575284.79106,-3682607.96396,4511038.02327,0.00007,0.00007,0.00009
3890871,712400477.55000,-2575284.85003,-3682607.93236,4511038.03698,0.00007,0.00007,0.00009
3890872,712400477.60000,-2575284.90925,-3682607.89903,4511038.04687,0.00007,0.00008,0.00009
3890873,712400477.65000,-2575284.96746,-3682607.86363,4511038.05163,0.00007,0.00008,0.00009


In [13]:
gps_pos_freed_trans = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/209/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/210/posfilter/POS_FREED_TRANS_TWTT',
    ],
)

gps_pos_freed_trans = gps_pos_freed_trans[
    ["time", "x", "y", "z", "xx", "yy", "zz",]
]
gps_pos_freed_trans

,time,x,y,z,xx,yy,zz
0,712215465.00000,-2575288.22500,-3682570.22700,4511064.31000,0.00076,0.00100,0.00089
1,712215467.26326,-2575286.78900,-3682571.52800,4511065.01800,0.00032,0.00055,0.00044
2,712215467.38340,-2575286.73500,-3682571.58200,4511065.01300,0.00041,0.00065,0.00054
3,712215467.40082,-2575286.72700,-3682571.59000,4511065.01000,0.00048,0.00072,0.00060
4,712215480.00000,-2575279.43900,-3682576.05400,4511065.58000,0.00076,0.00100,0.00089
...,...,...,...,...,...,...,...
47984,712396373.47335,-2575338.61000,-3682612.73300,4511002.67000,0.00050,0.00054,0.00064
47985,712396386.00000,-2575336.17700,-3682621.93600,4510996.73600,0.00076,0.00080,0.00089
47986,712396388.25092,-2575336.42700,-3682622.32600,4510995.56800,0.00031,0.00035,0.00044
47987,712396388.31850,-2575336.39400,-3682622.34700,4510995.57800,0.00038,0.00042,0.00051


In [14]:
common_df = kalman_filtered_df.merge(
    gps_pos_freed_trans,
    on="time",
    how="inner",
    suffixes=("_kalman", "_gold")
)

In [15]:
np.allclose(
    common_df["x_kalman"].values, common_df["x_gold"].values,
    rtol=0.0001
)

True

In [16]:
np.allclose(
    common_df["y_kalman"].values, common_df["y_gold"].values,
    rtol=0.0001
)

True

In [17]:
np.allclose(
    common_df["z_kalman"].values, common_df["z_gold"].values,
    rtol=0.0001
)

True

In [18]:
np.allclose(
    common_df["xx_kalman"].values, common_df["xx_gold"].values,
    rtol=0.1
)

False

In [ ]:
common_df.loc[10000:10010,["time", "xx_kalman", "xx_gold"]]

In [ ]:
common_df.loc[0:5,]